In [4]:
import pandas as pd
import numpy as np
class DataAug3D(object):
    
    def __init__(self, rotation=5, width_shift=0.01, height_shift=0.01, depth_shift=0.01, zoom_range=0.01,
                 rescale=1.1, horizontal_flip=True, vertical_flip=False, depth_flip=False):
        # define data preparation
        self.__datagen = ImageDataGenerator3D(rotation_range=rotation, width_shift_range=width_shift,
                                              height_shift_range=height_shift, depth_shift_range=depth_shift,
                                              zoom_range=zoom_range,
                                              rescale=rescale, horizontal_flip=horizontal_flip,
                                              vertical_flip=vertical_flip, depth_flip=depth_flip,
                                              fill_mode='nearest')

    def __ImageMaskTranform(self, images_path, index, number):
        # reshape to be [samples][depth][width][height][channels]
        imagesample = np.load(images_path)
        srcimages = np.zeros((imagesample.shape[0], imagesample.shape[1], imagesample.shape[2]))

        srcimage = imagesample.reshape([1, srcimages.shape[0], srcimages.shape[1], srcimages.shape[2], 1])
        i = 0
        for batch1, _ in self.__datagen.flow(srcimage, srcimage):
            i += 1
            batch1 = batch1[0, :, :, :, :]
            for j in range(batch1.shape[2]):
                npy_path = self.aug_path + str(index) + '_' + str(i) + ".npy"
                batchx = batch1.reshape([srcimages.shape[0], srcimages.shape[1], srcimages.shape[2]])
                np.save(npy_path, batchx)
            if i > number - 1:
                break

    def DataAugmentation(self, filepathX, number=5, aug_path=None):
        csvXdata = pd.read_csv(filepathX)
        dataX = csvXdata.iloc[:, :].values
        self.aug_path = aug_path
        for index in range(dataX.shape[0]):
            # For images
            images_path = dataX[index][0]
            self.__ImageMaskTranform(images_path, index, number)


In [5]:



import numpy as np
import scipy.ndimage as ndi
import os
import threading


def random_channel_shift(x, intensity, channel_axis=0):
    x = np.rollaxis(x, channel_axis, 0)
    min_x, max_x = np.min(x), np.max(x)
    channel_images = [np.clip(x_channel + np.random.uniform(-intensity, intensity), min_x, max_x)
                      for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x


def transform_matrix_offset_center(matrix, x, y, z):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    o_z = float(z) / 2 + 0.5
    offset_matrix = np.array([[1, 0, 0, o_x],
                              [0, 1, 0, o_y],
                              [0, 0, 1, o_z],
                              [0, 0, 0, 1]])
    reset_matrix = np.array([[1, 0, 0, -o_x],
                             [0, 1, 0, -o_y],
                             [0, 0, 1, -o_z],
                             [0, 0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix


def apply_transform(x, transform_matrix, channel_axis=0, fill_mode='nearest', cval=0.):
    x = np.rollaxis(x, channel_axis, 0)
    final_affine_matrix = transform_matrix[:3, :3]
    final_offset = transform_matrix[:3, 3]
    channel_images = [ndi.interpolation.affine_transform(x_channel, final_affine_matrix,final_offset, order=0, mode=fill_mode, cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x


def flip_axis(x, axis):
    x = np.asarray(x).swapaxes(axis, 0)
    x = x[::-1, ...]
    x = x.swapaxes(0, axis)
    return x


class ImageDataGenerator3D(object):
    def __init__(self,
                 featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=0.,
                 width_shift_range=0.,
                 height_shift_range=0.,
                 depth_shift_range=0.,
                 zoom_range=0.,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=False,
                 vertical_flip=False,
                 depth_flip=False,
                 rescale=None,
                 preprocessing_function=None,
                 dim_ordering='tf'):
        self.featurewise_center=featurewise_center
        self.samplewise_center=samplewise_center
        self.featurewise_std_normalization=featurewise_std_normalization
        self.samplewise_std_normalization=samplewise_std_normalization
        self.zca_whitening=zca_whitening
        self.rotation_range = rotation_range
        self.width_shift_range = width_shift_range
        self.height_shift_range = height_shift_range
        self.depth_shift_range = depth_shift_range
        self.zoom_range = zoom_range
        self.channel_shift_range = channel_shift_range
        self.fill_mode = fill_mode
        self.cval = cval
        self.horizontal_flip = horizontal_flip
        self.vertical_flip = vertical_flip
        self.depth_flip = depth_flip
        self.rescale = rescale
        self.preprocessing_function = preprocessing_function

        if dim_ordering not in {'tf', 'th'}:
            raise ValueError('dim_ordering should be "tf" (channel after row and '
                             'column) or "th" (channel before row and column). '
                             'Received arg: ', dim_ordering)
        self.dim_ordering = dim_ordering
        self.mean = None
        self.std = None
        self.principal_components = None
        if dim_ordering == 'tf':
            self.channel_axis = 4
            self.row_axis = 1
            self.col_axis = 2
            self.depth_axis = 3

        if np.isscalar(zoom_range):
            self.zoom_range = [1 - zoom_range, 1 + zoom_range]
        elif len(zoom_range) == 2:
            self.zoom_range = [zoom_range[0], zoom_range[1]]
        else:
            raise ValueError('zoom_range should be a float or '
                             'a tuple or list of two floats. '
                             'Received arg: ', zoom_range)  

    def flow(self, X, y=None, batch_size=32, shuffle=True, seed=None):
        return NumpyArrayIterator(
            X, y, self,
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            dim_ordering=self.dim_ordering)

    def standardize(self, x):
        # Only applied to X
        if self.rescale:
            x *= self.rescale
        # x is a single image, so it doesn't have image number at index 0
        img_channel_index = self.channel_axis - 1
        if self.samplewise_center:
            x -= np.mean(x, axis=img_channel_index, keepdims=True)
        if self.samplewise_std_normalization:
            x /= (np.std(x, axis=img_channel_index, keepdims=True) + 1e-7)

        if self.featurewise_center:
            x -= self.mean
        if self.featurewise_std_normalization:
            x /= (self.std + 1e-7)

        if self.zca_whitening:
            flatx = np.reshape(x, (x.size))
            whitex = np.dot(flatx, self.principal_components)
            x = np.reshape(whitex, (x.shape[0], x.shape[1], x.shape[2]))
        return x

    def random_transform(self, x,y):
        # Need to modify to transform both X and Y ----to do
        # x is a single image, so it doesn't have image number at index 0
        img_row_axis = self.row_axis - 1
        img_col_axis = self.col_axis - 1
        img_depth_axis = self.depth_axis - 1
        img_channel_axis = self.channel_axis - 1

        # use composition of homographies
        # to generate final transform that needs to be applied
        if self.rotation_range:
            theta_x = np.pi / 180 * np.random.uniform(-self.rotation_range, self.rotation_range)
            theta_y = np.pi / 180 * np.random.uniform(-self.rotation_range, self.rotation_range)
            theta_z = np.pi / 180 * np.random.uniform(-self.rotation_range, self.rotation_range)
        else:
            theta_x = 0
            theta_y = 0
            theta_z = 0
        rotation_matrix_x = np.array([[1, 0, 0, 0],
                                      [0, np.cos(theta_x), -np.sin(theta_x), 0],
                                      [0, np.sin(theta_x), np.cos(theta_x), 0],
                                      [0, 0, 0, 1]])
        rotation_matrix_y = np.array([[np.cos(theta_y), 0, np.sin(theta_y), 0],
                                      [0, 1, 0, 0],
                                      [-np.sin(theta_y), 0, np.cos(theta_y), 0],
                                      [0, 0, 0, 1]])
        rotation_matrix_z = np.array([[np.cos(theta_z), -np.sin(theta_z), 0, 0],
                                      [np.sin(theta_z), np.cos(theta_z), 0, 0],
                                      [0, 0, 1, 0],
                                      [0, 0, 0, 1]])
        if self.height_shift_range:
            tx = np.random.uniform(-self.height_shift_range, self.height_shift_range) * x.shape[img_row_axis]
        else:
            tx = 0
        if self.width_shift_range:
            ty = np.random.uniform(-self.width_shift_range, self.width_shift_range) * x.shape[img_col_axis]
        else:
            ty = 0
        if self.depth_shift_range:
            tz = np.random.uniform(-self.depth_shift_range, self.depth_shift_range) * x.shape[img_depth_axis]
        else:
            tz = 0

        translation_matrix = np.array([[1, 0, 0, tx],
                                       [0, 1, 0, ty],
                                       [0, 0, 1, tz],
                                       [0, 0, 0, 1]])

        if self.zoom_range[0] == 1 and self.zoom_range[1] == 1:
            zx, zy, zz = 1, 1, 1
        else:
            zx, zy, zz = np.random.uniform(self.zoom_range[0], self.zoom_range[1], 3)
        zoom_matrix = np.array([[zx, 0, 0, 0],
                                [0, zy, 0, 0],
                                [0, 0, zz, 0],
                                [0, 0, 0, 1]])

        transform_matrix = np.dot(np.dot(rotation_matrix_x, rotation_matrix_y), rotation_matrix_z)
        transform_matrix = np.dot(np.dot(transform_matrix, translation_matrix), zoom_matrix)

        h, w, d = x.shape[img_row_axis], x.shape[img_col_axis], x.shape[img_depth_axis]
        transform_matrix = transform_matrix_offset_center(transform_matrix, h, w, d)
        x = apply_transform(x, transform_matrix, img_channel_axis,
                            fill_mode=self.fill_mode, cval=self.cval)
        # For y,mask data,fill mode constant,cval =0
        y=apply_transform(y,transform_matrix,img_channel_axis, fill_mode=self.fill_mode, cval=self.cval)
        if self.channel_shift_range != 0:
            x = random_channel_shift(x,
                                     self.channel_shift_range,
                                     img_channel_axis)
        if self.horizontal_flip:
            if np.random.random() < 0.5:
                x = flip_axis(x, img_col_axis)
                y=flip_axis(y,img_col_axis)
        if self.vertical_flip:
            if np.random.random() < 0.5:
                x = flip_axis(x, img_row_axis)
                y=flip_axis(y,img_row_axis)
        if self.depth_flip:
            if np.random.random() < 0.5:
                x = flip_axis(x, img_depth_axis)
                y=flip_axis(y,img_depth_axis)
        return x,y


class Iterator(object):

    def __init__(self, n, batch_size, shuffle, seed):
        self.n = n
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.batch_index = 0
        self.total_batches_seen = 0
        self.lock = threading.Lock()
        self.index_generator = self._flow_index(n, batch_size, shuffle, seed)

    def reset(self):
        self.batch_index = 0

    def _flow_index(self, n, batch_size=32, shuffle=False, seed=None):
        # ensure self.batch_index is 0
        self.reset()
        while 1:
            if seed is not None:
                np.random.seed(seed + self.total_batches_seen)
            if self.batch_index == 0:
                index_array = np.arange(n)
                if shuffle:
                    index_array = np.random.permutation(n)

            current_index = (self.batch_index * batch_size) % n
            if n >= current_index + batch_size:
                current_batch_size = batch_size
                self.batch_index += 1
            else:
                current_batch_size = n - current_index
                self.batch_index = 0
            self.total_batches_seen += 1
            yield (index_array[current_index: current_index + current_batch_size],
                   current_index, current_batch_size)

    def __iter__(self):
        # needed if we want to do something like:
        # for x, y in data_gen.flow(...):
        return self

    def __next__(self, *args, **kwargs):
        return self.next(*args, **kwargs)


class NumpyArrayIterator(Iterator):

    def __init__(self, x, y, image_data_generator,
                 batch_size=32, shuffle=False, seed=None,
                 dim_ordering='tf'):
        if y is not None and len(x) != len(y):
            raise ValueError('X (images tensor) and y (labels) '
                             'should have the same length. '
                             'Found: X.shape = %s, y.shape = %s' %
                             (np.asarray(x).shape, np.asarray(y).shape))
        self.x = np.asarray(x, dtype=np.float32)
        if self.x.ndim != 5:
            raise ValueError('Input data in `NumpyArrayIterator` '
                             'should have rank 5. You passed an array '
                             'with shape', self.x.shape)
        channels_axis = 4 if dim_ordering == 'tf' else 1
        if self.x.shape[channels_axis] not in {1, 3, 4}:
            raise ValueError('NumpyArrayIterator is set to use the '
                             'dimension ordering convention "' + dim_ordering + '" '
                             '(channels on axis ' + str(channels_axis) + '), i.e. expected '
                             'either 1, 3 or 4 channels on axis ' + str(channels_axis) + '. '
                             'However, it was passed an array with shape ' + str(self.x.shape) +
                             ' (' + str(self.x.shape[channels_axis]) + ' channels).')
        if y is not None:
            self.y = np.asarray(y)
        else:
            self.y = None
        self.image_data_generator = image_data_generator
        self.dim_ordering = dim_ordering
        super(NumpyArrayIterator, self).__init__(x.shape[0], batch_size, shuffle, seed)

    def next(self):
        # for python 2.x.
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch
        # see http://anandology.com/blog/using-iterators-and-generators/
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros(tuple([current_batch_size] + list(self.x.shape)[1:]))
        batch_y = np.zeros(tuple([current_batch_size] + list(self.y.shape)[1:]))
        for i, j in enumerate(index_array):
            x = self.x[j]
            
            label = self.y[j]
            x,label = self.image_data_generator.random_transform(x.astype(np.float32),label.astype(np.float32))
            x=self.image_data_generator.standardize(x)
            batch_x[i] = x
            batch_y[i]=label
        return batch_x, batch_y

In [6]:
aug = DataAug3D(rotation=45, width_shift=0.05, height_shift=0.05, depth_shift=0, zoom_range=0)
aug.DataAugmentation(r'D:\LUNA16\classsification2\Train_X.csv', 5, aug_path=r'D:\LUNA16\classsification2\aug2\aug')

In [ ]:
augmented_path = r'D:\LUNA16\classsification\aug'
aug_file_names = []
for file in os.listdir(augmented_path):
    aug_file_names.append(os.path.join(augmented_path, file))

In [ ]:
aug_file_names = pd.DataFrame(aug_file_names)

In [ ]:
aug_file_names.to_csv('D:\LUNA16\classsification\aug.csv')